## A Gentle Introduction to Recommender Systems with Implicit Feedback


https://nbviewer.jupyter.org/github/jmsteinw/Notebooks/blob/master/RecEngine_NB.ipynb

### Part1: introdiction

In [2]:
from IPython.display import YouTubeVideo
YouTubeVideo('bLhq63ygoU8')

##### Content Based (Pandora)
In the case of Pandora, the online streaming music company, they decided to engineer features from all of the songs in their catalog as part of the Music Genome Project. Most songs are based on a feature vector of approximately 450 features, which were derived in a very long and arduous process. Once you have this feature set, one technique that works well enough is to treat the recommendation problem as a binary classification problem. This allows one to use more traditional machine learning techniques that output a probability for a certain user to like a specific song based on a training set of their song listening history. Then, simply recommend the songs with the greatest probability of being liked.

Most of the time, however, you aren't going to have features already encoded for all of your products. This would be very difficult, and it took Pandora several years to finish so it probably won't be a great option.

##### Demographic Based (Facebook)
If you have a lot of demographic information about your users like Facebook or LinkedIn does, you may be able to recommend based on similar users and their past behavior. Similar to the content based method, you could derive a feature vector for each of your users and generate models that predict probabilities of liking certain items.

Again, this requires a lot of information about your users that you probably don't have in most cases.

So if you need a method that doesn't care about detailed information regarding your items or your users, collaborative filtering is a very powerful method that works with surprising efficacy.

### Part2:Processing the Data

In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve


In [2]:
website_url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'
retail_data = pd.read_excel(website_url)

In [3]:
retail_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


#### Description
- invocieNo : invoice number for different purchases
- StockCode : item ID
- Description : item description
- Quantity : the number purchased
- InvoiceDate : the date of purchase
- UnitPrice : the price of the items
- CustomerID : a customer ID
- Country : the country of origin for the customer.

In [4]:
retail_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


CustomerID에서 missing value를 찾을 수 있다. customerID가 없으면 특정 사용자와 아이템 간의 매칭이 불가능 하기 때문에 지우고, 값이 있는 데이터로만 추천을 진행한다.

In [4]:
cleaned_retail = retail_data.loc[pd.isnull(retail_data.CustomerID)==False]

In [6]:
cleaned_retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      406829 non-null object
StockCode      406829 non-null object
Description    406829 non-null object
Quantity       406829 non-null int64
InvoiceDate    406829 non-null datetime64[ns]
UnitPrice      406829 non-null float64
CustomerID     406829 non-null float64
Country        406829 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.9+ MB


rating matrix를 만들기 전에, 각 아이템과 아이템의 description을 이어주는 표를 하나 만들어 놓으면 보기 편할 것이다

In [5]:
item_lookup = cleaned_retail[['StockCode','Description']].drop_duplicates()
item_lookup['StockCode'] = item_lookup.StockCode.astype(str)

In [8]:
item_lookup.head()

,StockCode,Description
0,85123A,WHITE HANGING HEART T-LIGHT HOLDER
1,71053,WHITE METAL LANTERN
2,84406B,CREAM CUPID HEARTS COAT HANGER
3,84029G,KNITTED UNION FLAG HOT WATER BOTTLE
4,84029E,RED WOOLLY HOTTIE WHITE HEART.


What we need to:

- Group purchase quantities together by stock code and item ID
- Change any sums that equal zero to one (this can happen if items were returned, but we want to indicate that the user actually purchased the item instead of assuming no interaction between the user and the item ever took place)
- Only include customers with a positive purchase total to eliminate possible errors
- Set up our sparse ratings matrix


In [6]:
cleaned_retail['CustomerID'] = cleaned_retail.CustomerID.astype(int)
cleaned_retail = cleaned_retail[['StockCode','Quantity', 'CustomerID']]
grouped_cleaned = cleaned_retail.groupby(['CustomerID', 'StockCode']).sum().reset_index()
grouped_cleaned.Quantity.loc[grouped_cleaned.Quantity == 0] = 1 #곱해질때 값이 0이 되는 것을 막기 위해
grouped_purchased = grouped_cleaned.query('Quantity > 0')

/home/yeoyoung/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/yeoyoung/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [10]:
grouped_purchased.head()

,CustomerID,StockCode,Quantity
0,12346,23166,1
1,12347,16008,24
2,12347,17021,36
3,12347,20665,6
4,12347,20719,40


explicting rating을 사용하는 대신에, 구매량은 얼마나 유저와 강한 interection을 하는지 알려주는 일종의 'confidence'를 나타낸다.
많은 양을 구매할수록 matrix of purchases에 더 많은 weight를 가지게 된다. 

In [7]:
customers = list(np.sort(grouped_purchased.CustomerID.unique()))
products = list(grouped_purchased.StockCode.unique())
quantity = list(grouped_purchased.Quantity)
                 
rows = grouped_purchased.CustomerID.astype('category', categories = customers).cat.codes #R의 factor
cols = grouped_purchased.StockCode.astype('category', categories = products).cat.codes
purchases_sparse = sparse.csr_matrix((quantity, (rows,cols)), shape=(len(customers), len(products)))

/home/yeoyoung/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  """
/home/yeoyoung/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  


In [12]:
purchases_sparse

<4338x3664 sparse matrix of type '<class 'numpy.int64'>'
	with 266723 stored elements in Compressed Sparse Row format>

4338명의 고객, 3664개의 아이템, 266723개가 구매(=user/item interaction) 되었다. 
<br/>
행렬의 sparsity계산은 다음과 같다

In [8]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] #number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0])
sparsity = 100*(1-(num_purchases/matrix_size))
sparsity

98.32190920694744

Collaborative filtering을 위해서 maximum sparsity는 99.5% 정도이다.따라서 우리는 그 다음과정을 충분히 진행할 수 있다.

### Part3 : Creating a Training and Validation Set

![image.png](https://nbviewer.jupyter.org/github/jmsteinw/Notebooks/blob/master/Traintest_ex.png)

하지만 CF에서는 모든 유저와 아이템의 인터렉션이 있어야 위에처럼 할 수 있기때문에 적절하지 않다. 
좋은 방법은 무작위로 선택한 training 단계 동안 모델에서 user-item interaction의 일정 비율을 숨기는 것이다. 
그런 다음, test 단계에서 유저가 추천된 품목 중 실제로 구매에 성공한 것이 몇 개나 되는지 확인한다. 
이상적으로, 당신은 궁극적으로 어떤 종류의 A/B 시험으로 당신의 추천를 test하거나 일정 시간 이후의 데이터가 test에 사용되는 동안 training 하는데 사용되는 시계열 데이터를 활용한다.

![img](https://nbviewer.jupyter.org/github/jmsteinw/Notebooks/blob/master/MaskTrain.png)

In [9]:
import random


In [10]:
def make_train (ratings, pct_test = 0.2):
    
    '''
original user-item matrix와 "mask"에서 user-item interaction이 testset으로 사용되었을 때 original rating의 percentage를 가진다.


test set : 모든 original rating이 포함
train set: 원래 rating 매트릭스에서 지정된 percentage인 0으로 대체한다.

parameters: 

rating - train/test set를 생성하고자 하는 original rating matrix. sparse한 csr_matrix의 일종이다. 

pct_test - test set 과 나중에 비교하기 위한 trainig set에서 masking하고자 하는 interaction이 발생한 user-item interaction 의 percentage.

returns:

training_set - 원래 interaction을 0으로 설정한 user-item 쌍의 일정 비율을 가진 원본 데이터의 변경 버전

test_set - rating 순서가 실제 interaction과 어떻게 비교되는지를 볼 수 있도록 원래의 rating 매트릭스의 사본.

user_inds - AUC를 통해 성능을 평가할 때 나중에 필요할 것.
'''

    test_set = ratings.copy()
    test_set[test_set!=0] = 1 #binary 로 바꾸기 위해
    training_set = ratings.copy()
    nonzero_inds = training_set.nonzero()
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1]))
    random.seed(0)
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) #np.ceil : 올림
    samples = random.sample(nonzero_pairs, num_samples)
    user_inds = [index[0] for index in samples]
    item_inds = [index[1] for index in samples]
    
    training_set[user_inds, item_inds] = 0 #선택된거는 다 0으로 만들어 주기
    training_set.eliminate_zeros()
    return training_set, test_set, list(set(user_inds))

training set, 0(구매x)/1(구매) 로 binary화 된 test set. 적어도 한 아이템을 가린 아이템 리스트가 return 된다.
우리는 이들에게만 recommender system을 테스트 할것이고 여기서는 지금 uner-item interaction의 20%를 masking 하고 있다.

In [11]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test=0.2)

여기까지 train/test를 split했다. 

### Part4 : Implementing ALS for Implict Feedback

[참고논문]
[Hu, Koren, and Volinsky](http://yifanhu.net/PUB/cf.pdf)

우리가 가지고 있는 sparse한 rating matrix를 confidence matrix로 바꿔야 한다. 


*C<sub>ui</sub>=1+αr<sub>ui</sub>*

C<sub>ui</sub> : 유저u와 아이템i의 confidence matrix
α :rating 선호도의 linear scaling
r<sub>ui</sub> : original purchase matrix

일반적으로 40정도의 dimension을 good starting point 라고 본다. 

최종 계산식은 아래의 두 식을 반복할 것이다. 

- *x<sub>u</sub>=(Y<sup>T</sup>Y+Y<sup>T</sup>(C<sup>u</sup>-I)Y+λI)<sup>-1</sup>Y<sup>T</sup>C<sup>u</sup>p(u)*

- *y<sub>i</sub>=(X<sup>T</sup>X+X<sup>T</sup>(C<sup>i</sup>-I)X+λI)<sup>-1</sup>X<sup>T</sup>C<sup>i</sup>p(i)*

In [12]:
def implict_weighted_ALS(training_set, lambda_val = 0.1, alpha = 40, iterations = 10, rank_size =20, seed = 0 ):
    '''

매개 변수:

training_set - feature m x n의 rating 매트릭스, 여기서 m은 사용자 수, n은 품목 수.sparse csr 매트릭스가 되어야 한다. 

lambda_val - for normalization. 이 값을 증가시키면 bias은 증가되지만 variance은 감소한다. 기본값은 0.1이다. 

alpha - 논문에서 논한 confidence matrix와 관련된 파라미터, 여기서 Cui = 1 + alpha*Rui. 논문은 가장 효과적인 default값으로 40. 이를 줄이면 각종 rating간의 confidence뢰도의 변동성이 줄어든다.

iterations - user feature vector와 item feature vector가 최소 제곱을 alterate하는 횟수. 더 많은 반복은 증가된 계산의 비용으로 더 나은 융합을 가능하게 할 것이다. 저자들은 10번 반복하면 충분하지만 수렴하려면 더 많은 것이 필요할 수 있다는 것을 발견했다. 

rank_size - user/item feature vector의 latent feature의 수. 논문은 이것을 20-200 사이에서 변화시킬 것을 권고했다. feature의 수를 늘리면 overfitting할 수 있지만 bias을 줄일 수 있다. 

seed - 재현 가능한 결과를 위해 시드를 설정

returns:

user 및 item 에 대한 feature vector. 이러한 feature vector의 dot product는 원래 매트릭스의 각 지점에서 예상되는 "rating"을 제공해야 한다. 
'''
    # first set up our confidence matrix
    
    conf = (alpha*training_set) #1은 나중에 더할 것
    
    num_user = conf.shape[0]
    num_item = conf.shape[1] #original rating matrix의 size (m by n)
    
    #initialize our X/Y feature vectors randomly with a set seed
    rstate = np.random.RandomState(seed)
    
    X = sparse.csr_matrix(rstate.normal(size = (num_user, rank_size)))
    Y = sparse.csr_matrix(rstate.normal(size = (num_item, rank_size)))
    
    X_eye = sparse.eye(num_user)
    Y_eye = sparse.eye(num_item)

    lambda_eye = lambda_val * sparse.eye(rank_size) #regularization term
    
    # we can compute this before iteration starts.
    
    # Begin iterations
    
    for iter_step in range(iterations): #y를 고정하고 x풀고, x 고정하고 y풀고 반복
        yty = Y.T.dot(Y)
        xtx = X.T.dot(X)
        
        #y를 고정시키고 x를 푸는 과정
        for u in range(num_user):
            conf_samp = conf[u,:].toarray() #confidence matrix에서 user row하나 골라서 array로 변환
            pref = conf_samp.copy()
            pref[pref != 0] = 1 #binary preference vector 생성
            cui = sparse.diags(conf_samp ,[0]) #1을 더하지 않은(cui-I) 부분
            ytcuiy = Y.T.dot(cui).dot(Y)
            ytcupu = Y.T.dot(cui + Y_eye).dot(pref.T) #ytcupu
            X[u] = spsolve(yty + ytcuiy + lambda_eye, ytcupu)
             # Solve for Xu = ((yTy + yT(Cu-I)Y + lambda*I)^-1)yTCuPu,
                
        for i in range(num_item):
            conf_samp = conf[:,i].T.toarray()
            pref = conf_samp.copy()
            pref[pref != 0] = 1
            cii = sparse.diags(conf_samp,[0])
            xtciix = X.T.dot(cii).dot(X)
            xtcipi = X.T.dot(cii + X_eye).dot(pref.T)
            Y[i] = spsolve(xtx + xtciix + lambda_eye, xtcipi)

    return X, Y.T  # Transpose at the end to make up for not being transposed at the beginning. 
                  # Y needs to be rank x n. Keep these as separate matrices for scale reasons. 

   


        
        
    

In [13]:
user_vecs, item_vecs = implict_weighted_ALS(product_train, lambda_val = 0.1, alpha = 15, iterations = 1, rank_size = 20)

user vector 와 item vector 의 내적을 통해 특정 유저의 rating을 알 수 있다.
- ex) 첫 번째 유저

In [24]:
user_vecs[0,:].dot(item_vecs).toarray()[0,:5]

array([ 0.01218043, -0.00251753,  0.00943813,  0.00054114,  0.02303351])

3664개의 stock중 첫번째 5개의 item의 sample이다. 첫번째 유저의 한 번 실행시 15분 정도 걸리며, 최소 10번 이상 반복해야 한다.

### Speeding Up ALS

Ben Frederickson 이란 분이 parallelizing version의 ALS를 고안해내셨다.(훨씬 빠르다)

In [14]:
import implicit

- 알파에 대한 매개변수 x, 
- matrix type을 double로 설정했는지 확인 필요

In [15]:
alpha = 15
user_vecs, item_vecs = implicit.alternating_least_squares((product_train*alpha).astype('double'),
                                                         factors=20, regularization=0.1,
                                                         iterations =50)

This method is deprecated. Please use the AlternatingLeastSquares class instead
100%|██████████| 50.0/50 [00:00<00:00, 88.67it/s]


이제 모든 user와 item에 대한 recommendation 을 가지고 있다. 이게 좋은지 어떻게 알 수 있을까?

### Evaluating the Recommender System

일전에 training set 의 20%를 가려놓았는데, 이를 추천시스템의 성능 평가에 사용한다. 
일반적으로 ROC curve를 사용한다 (면적이 넓으면 잘 되고 있다는 의미)
 * 주로 ROC curve 는 좋다/아니다 와 같은 이분법적인 평가에 적합



따라서 한 개 이사으이 masked item이 있다면, auc의 mean area를 구할 수 있다.
만약 간단한 추천시스템으로 제일 인기있는 아이템을 추천했다면 auc의 평균도 계산해야 한다.


auc를 계산해 보자.

In [16]:
from sklearn import metrics

In [17]:
def auc_score (predictions, test):
    '''
    curve 아래쪽의 면적 계산
    
    parameters: 
    
    prediction - 예측결과
    test - 현재 비교하고 있는 실제 목표 결과
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)

유저가 비교할 수 있는 가장 인기 있는 아이템에 대한 auc를 계산해야 한다.

In [18]:
def calc_mean_auc (training_set, altered_users, predictions, test_set):
    '''
    평균 auc 계산
    
    parameters:
    
    training_set - make_train에서 만들어지는 training set. user-item interaction 은 0으로 리셋됨.
    altered_users - make_train 함수에서 user/item 쌍을 하나 이상 변경한 유저 수치
    predictions - 암묵적 MF로부터의 output. user/item 쌍에 대한 예측 rating matrix. 
    test_set - make_train 에서 만들어진 test_set
    
    '''
    
    store_auc = [] #auc들을 저장할 빈 리스트
    popularity_auc = []
    pop_items = np.array(test_set.sum(axis=0)).reshape(-1) #1차원 배열
    item_vecs = predictions[1]
    for user in altered_users:
        training_row = training_set[user,:].toarray().reshape(-1) #1차원 배열
        zero_inds = np.where(training_row == 0)
        
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        
        # Get only the items that were originally zero
        # 원래 반복이 없었던 user에 대한 MF prediction에서 모든 rating 선택
        
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1)
        
        pop = pop_items[zero_inds] #get item popularity
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
        
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc)) #소숫점 3째 자리 반올림
        
        
                                                      

우리는 이제 추천시스템이 어떻게 돌아가고 있는지 볼 수 있다. 
ALS 함수에서 csr_matrix 형식과 item vector들을 변환해야 한다.

In [19]:
calc_mean_auc(product_train, product_users_altered,[sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)

(0.871, 0.814)

mean auc = 0.871, popular item benchmark = 0.814
hyterparameter들을 조정하여 auc점수를 조정할 수 있다.

### A Recommendation Example

우리의 추천시스템을 통해 benchmark popularity를 넘었다는 것을 증명했다. 즉 auc 0.87의 의미는 실제로 추천시스템에서 유저기 test set에서 구매한 아이템을 유저가 구매하지 않은 아이템보다 훨씬 더 자주 추천하고 있음을 의미한다. 
어떻게 작동하는지 보기위해서 특정 유저에게 우어진 추천안을 확인해보고 그것이 얼마나 make sence한지 확인해 보자.

In [20]:
customers_arr = np.array(customers) #rating matrix에서의 customer id 배열
products_arr = np.array(products) #rating matrix에서의 product id 배열

In [21]:
def get_items_purchased (customer_id, mf_train, customers_list, products_list, item_lookup):
    
    '''   
training set 에서 특정 유저가 이미 구매한 아이템들

parameters: 

customer_id - 최소 한 번 이상 이전에 구입한 고객의 ID
mf_train - 사용된 초기 rating matrix set (without weights applied)
customer_list - rating matrix에 사용된 customer의 배열
products_list - rating matrix에 사용된 product의 배열
item_lookup - 사용 가능한 고유 아이템ID/아이템 설명의 간단한 데이터 프레임

return:
training set에서 이미 구매한 특정 유저의 아이템id와 아이템 설명 목록

    '''
    cust_ind =  np.where(customers_list == customer_id)[0][0] #customer id 의 row index
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] #구매 아이템의 column index
    prod_codes = products_list[purchased_ind] #구매한 아이템의 stock code
    
    return item_lookup.loc[item_lookup.StockCode.isin(prod_codes)]


In [22]:
customers_arr[:5]

array([12346, 12347, 12348, 12349, 12350])

12346 id를 가지고 있는 첫 번째 고객이 구입한 앙이템을 살펴보자

In [23]:
get_items_purchased(12346, product_train, customers_arr, products_arr, item_lookup )

,StockCode,Description
61619,23166,MEDIUM CERAMIC TOP STORAGE JAR


medium 사이즈의 ceramic jar for storage를 산 것을 알 수 있다. 
추천 시스템에서는 이 고객이 어떤 아이템을 구매해야한다고 하는가?

In [24]:
from sklearn.preprocessing import MinMaxScaler

In [41]:
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10 ):
    
    '''
      
가장 추천하는 아이템 유저에게 return

parameters:

customer_id - 최소 한 번 이상 이전에 구입한 고객의 ID
mf_train - 사용된 초기 rating matrix set (without weights applied)
user_vecs - fitted matrix factorization 에서의 user vector
item_vecs - fitted matrix facotrization 에서의 item vector
customer_list - rating matrix에 사용된 customer의 배열 (in order of matrix)
products_list - rating matrix에 사용된 product의 배열 (in order of matrix)
item_lookup - 사용 가능한 고유 아이템ID/아이템 설명의 간단한 데이터 프레임
num_items - 구매하지 않은 아이템들 중 top N개의 추천 아이템의의 수. 기본값은 10 

return:

- user/item vector을 기준을 선택된 아이템 중 상위 N개 추천
    '''
    
    cust_ind = np.where(customer_list == customer_id)[0][0] #customer id의 index row
    pref_vec = mf_train[cust_ind, :].toarray() #training set rating matrix에서 rating 가져오기
    pref_vec = pref_vec.reshape(-1) + 1 #1을 더하면 아직 구입하지 않은 아이템들이 1로 표시
    pref_vec[pref_vec>1] = 0 #이미 구매한 것들을 0으로 지정
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) #user vecot와 모든 item vector와의 내적
    
    #0~1사이로 scailing
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0]
    recommend_vector = pref_vec*rec_vector_scaled
    
    #구매된 아이템에는 다시 0을 곱해줌
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] #순서대로 정렬
    
    
    #best recommendations
    rec_list=[]
    for index in product_idx:
        code = item_list[index]
        rec_list.append([code, item_lookup.Description.loc[item_lookup.StockCode == code].iloc[0]])
        
    codes = [item[0] for item in rec_list]
    descriptions = [item[1] for item in rec_list]
    
    final_frame = pd.DataFrame ({'StockCode':codes, 'Description':descriptions})
    return final_frame[['StockCode','Description']]
    
        
    

이미 구매한 상품은 유저에게 추천하지 않는다. 유저가 무엇을 선택하기로 결정하는지 확인해보자.

In [42]:
rec_items(12346, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup, num_items = 10)

,StockCode,Description
0,23167,SMALL CERAMIC TOP STORAGE JAR
1,23165,LARGE CERAMIC TOP STORAGE JAR
2,22980,PANTRY SCRUBBING BRUSH
3,22963,JAM JAR WITH GREEN LID
4,22978,PANTRY ROLLING PIN
5,23295,SET OF 12 MINI LOAF BAKING CASES
6,23294,SET OF 6 SNACK LOAF BAKING CASES
7,22982,PANTRY PASTRY BRUSH
8,23296,SET OF 6 TEA TIME BAKING CASES
9,23293,SET OF 12 FAIRY CAKE BAKING CASES


구매이력으로만 판단한 것 치고 괜찮은 결과가 나온다.
- 이미 구입한 사람에게 다른 사이즈의 ceramic jar 추천
- ceramic jar과 사용이 비슷한 제품 추천 (jar magnets, suger dispenser)


대량 구매를 하지 않은 다른 유저에게도 시도해보자.

In [43]:
get_items_purchased(12353,product_train, customers_arr, products_arr, item_lookup)

#

,StockCode,Description
2148,37446,MINI CAKE STAND WITH HANGING CAKES
2149,37449,CERAMIC CAKE STAND + HANGING CAKES
4859,37450,CERAMIC CAKE BOWL + HANGING CAKES
5108,22890,NOVELTY BISCUITS CAKE STAND 3 TIER


In [44]:
rec_items(12353, product_train, user_vecs, item_vecs, customers_arr,
         products_arr, item_lookup, num_items=10)

,StockCode,Description
0,21231,SWEETHEART CERAMIC TRINKET BOX
1,22645,CERAMIC HEART FAIRY CAKE MONEY BANK
2,37447,CERAMIC CAKE DESIGN SPOTTED PLATE
3,37448,CERAMIC CAKE DESIGN SPOTTED MUG
4,22055,MINI CAKE STAND HANGING STRAWBERY
5,72741,GRAND CHOCOLATECANDLE
6,22646,CERAMIC STRAWBERRY CAKE MONEY BANK
7,22644,CERAMIC CHERRY CAKE MONEY BANK
8,21232,STRAWBERRY CERAMIC TRINKET BOX
9,22059,CERAMIC STRAWBERRY DESIGN MUG


ceramic items 와 cake를 테마로 추천해 주는 것을 확인할 수 있다. 

In [45]:
get_items_purchased(12361,product_train, customers_arr, products_arr, item_lookup)


,StockCode,Description
34,22326,ROUND SNACK BOXES SET OF4 WOODLAND
35,22629,SPACEBOY LUNCH BOX
37,22631,CIRCUS PARADE LUNCH BOX
93,20725,LUNCH BAG RED RETROSPOT
369,22382,LUNCH BAG SPACEBOY DESIGN
547,22328,ROUND SNACK BOXES SET OF 4 FRUITS
549,22630,DOLLY GIRL LUNCH BOX
1241,22555,PLASTERS IN TIN STRONGMAN
58132,20725,LUNCH BAG RED SPOTTY


In [46]:
rec_items(12361, product_train, user_vecs, item_vecs, customers_arr,
         products_arr, item_lookup, num_items=10)

,StockCode,Description
0,22662,LUNCH BAG DOLLY GIRL DESIGN
1,22551,PLASTERS IN TIN SPACEBOY
2,20726,LUNCH BAG WOODLAND
3,84997D,PINK 3 PIECE POLKADOT CUTLERY SET
4,20727,LUNCH BAG BLACK SKULL.
5,22383,LUNCH BAG SUKI DESIGN
6,20719,WOODLAND CHARLOTTE BAG
7,20728,LUNCH BAG CARS BLUE
8,23206,LUNCH BAG APPLE DESIGN
9,20724,RED RETROSPOT CHARLOTTE BAG


가방과 점심관련 많은 아이템들이 추천된 것을 확인할 수 있다.

### Summary

- implict feedback으로 추천시스템을 설계, 추천제공, 추천 테스트 하는 방법

rating matrix의 크기가 너무 크다면 spark를 사용하는 것도 방법이다
만약 추천시스템을 업그레이드 하고 싶다면, 구매 이력과 유저/아이템 정보를 통합하는 하이브리드 시스템이 가장 좋을 것이다.
